# Comparison of SB3 Soft Actor-Critic CTCE trained agents with different reward functions
### Trained for 3.960.000 steps (720 (episode) x 5500)

In [76]:
import numpy as np 
import pandas as pd

import math
import sys
import os

import gym
gym.__version__

from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import TensorBoardOutputFormat

from training_simple_policy.custom_reward import CustomReward

import seaborn as sns
import matplotlib.pyplot as plt

### Initialize environment

In [26]:
def makeEnv():# schema path
    schema_path = os.path.join("./training_simple_policy/data/", "schema_edited.json")

    # create environment
    env = CityLearnEnv(schema=schema_path, reward_function=CustomReward, central_agent=True)

    # wrap environment for use in stablebaselines3
    env = NormalizedObservationWrapper(env)
    env = StableBaselines3Wrapper(env)

    return env

env = makeEnv()

### Load models

In [27]:
# Given reward functions
model_energy_reward = SAC.load("habrok_training_results/training_energy_reward3/models/energy_reward_SAC.zip")
model_energy_reward.set_env(env)

model_comfort_reward = SAC.load("habrok_training_results/training_comfort_reward3/models/comfort_reward_SAC.zip")
model_comfort_reward.set_env(env)

model_solar_reward = SAC.load("habrok_training_results/training_solar_reward3/models/solar_reward_SAC.zip")
model_solar_reward.set_env(env)

model_solarcomfort_reward = SAC.load("habrok_training_results/training_solarcomfort_reward3/models/solarcomfort_reward_SAC.zip")
model_solarcomfort_reward.set_env(env)

# Custom reward function
# model_custom_reward = SAC.load("habrok_training_results/training_comfort_reward3/models/custom_reward_SAC.zip")
# model_custom_reward.set_env(env)

/home/hkolstee/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


### Evaluate

In [64]:
def evaluate(model, env, name):
    observations = env.reset()

    while not env.done:
        actions, _ = model.predict(observations, deterministic=True)
        observations, _, _, _ = env.step(actions)
    
    kpis = env.evaluate()

    kpis = kpis.pivot(index='cost_function', columns='name', values='value')
    kpis = kpis.dropna(how='all')
    kpis.insert(0, "Reward function", name)

    return kpis

In [65]:
kpis_energy = evaluate(model_energy_reward, env, "Energy reward")
kpis_comfort = evaluate(model_comfort_reward, env, "Comfort reward")
kpis_solar = evaluate(model_solar_reward, env, "Solar reward")
kpis_solarcomfort = evaluate(model_solarcomfort_reward, env, "Comfort+solar reward")
# kpis_custom = evaluate(model_custom_reward, env, "Custom reward")

In [93]:
kpis = pd.DataFrame()

kpis = kpis.append(kpis_energy)
kpis = kpis.append(kpis_comfort)
kpis = kpis.append(kpis_solar)
kpis = kpis.append(kpis_solarcomfort)
# kpis.append(kpis_custom)

pd.set_option('display.max_rows', 1000)
kpis = kpis.sort_values(["cost_function", "Reward function"])



# display(kpis)
display(kpis.loc[("'annual_peak_average','Comfort reward', 'District'")])
# display(kpis.loc["cost_function"] == "annual_normalized_unserved_energy_total")


KeyError: "'annual_peak_average','Comfort reward', 'District'"